# DD_NMROM
Notebook to implement and test DD NM-ROM on time-dependent 2D Burgers' equation.  
Author: Alejandro Diaz  
Date:  1/30/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy.sparse as sp
# import scipy.linalg as la
# from time import time
from utils.Burgers2D_probgen import Burgers2D
from utils.domain_decomposition import DD_FOM, DDFOM_data
from utils.lsrom import compute_bases_from_svd, assemble_snapshot_matrix
from IPython.display import Image
from utils.nmrom import DD_NMROM, dd_rbf
# from time import time
import sys, os, torch
import dill as pickle

/usr/workspace/diaz64/anaconda/envs/opence-1.7.2-cuda-11.4/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
plt.rc('font', size=20)
plt.rcParams['lines.linewidth'] = 4
plt.rcParams['text.usetex'] = True

In [3]:
# make directories for figures and data
data_dir = './data/'
fig_dir0 = './figures/'
for d in [data_dir, fig_dir0]:
    if not os.path.exists(d): os.mkdir(d)

## Set FOM parameters, initial condition

In [4]:
# parameters for physical domain and FD discretization
x_lim = [0, 1]
y_lim = [0, 1]
nx, ny    = 60, 60
viscosity = 1e-2

# number of subdomains in x and y directions
nsub_x, nsub_y = 2, 1
nsub = nsub_x*nsub_y

# time integration parameters
nt    = 1000
t_lim = [0, 2]
ht    = (t_lim[1]-t_lim[0])/nt

# parameter for initial condition
mu = 1.0

scaling = -1

In [5]:
# parameterized initial conditions
def u0(XY, mu):
    val = np.zeros(len(XY))
    for i, xy in enumerate(XY):
        if np.all([xy[0] >= 0.0, xy[0] <= 0.5, xy[1] >= 0.0, xy[1] <= 0.5]):
            val[i] = mu*np.sin(2*np.pi*xy[0])*np.sin(2*np.pi*xy[1])
    return val 

def v0(XY, mu):
    val = np.zeros(len(XY))
    for i, xy in enumerate(XY):
        if np.all([xy[0] >= 0.0, xy[0] <= 0.5, xy[1] >= 0.0, xy[1] <= 0.5]):
            val[i] = mu*np.sin(2*np.pi*xy[0])*np.sin(2*np.pi*xy[1])
    return val 

## Initialize monolithic FOM and DD FOM

In [6]:
# initialize model
print('Initializing Burgers model...')
sys.stdout.flush()
fom = Burgers2D(nx, ny, x_lim, y_lim, viscosity)
print('Done!')

Initializing Burgers model...
Done!


In [7]:
# initialize DD FOM
ddfom = DD_FOM(fom, nsub_x, nsub_y, scaling=scaling)
ddfom.set_initial(lambda xy: u0(xy, mu), lambda xy: v0(xy, mu))

In [8]:
# march=2
# u_full, v_full, u_intr, v_intr, u_intf, v_intf, lam, runtime, flag = ddfom.solve([0, 2/nt*march],
#                                                                            march, 
#                                                                            tol=1e-4,
#                                                                            maxit=20, 
#                                                                             print_hist=True)

## Load FOM solution

In [9]:
ddfom_data = DDFOM_data(nx, ny, nt, viscosity, mu, nsub_x, nsub_y, data_dir)
ddfom_data.assemble_full_solution(ddfom)

## Build DD NM-ROM

In [10]:
# parameters for NM-ROM components
intr_size   = [8]*ddfom.n_sub#[5, 5]
intr_rnnz   = [5]*ddfom.n_sub#[5, 5]
intr_rshift = [5]*ddfom.n_sub#[5, 5]
intr_act    = ['Swish']*ddfom.n_sub#['Swish', 'Swish']
intr_loss   = ['RelMSE']*ddfom.n_sub#['AbsMSE', 'AbsMSE']
intr_batch  = [32]*ddfom.n_sub#[32, 32]
wd_intr     = 1e-6

intf_size   = [2]*ddfom.n_sub#[5, 5]
intf_rnnz   = [5]*ddfom.n_sub#[5, 5]
intf_rshift = [5]*ddfom.n_sub#[5, 5]
intf_act    = ['Swish']*ddfom.n_sub#['Swish', 'Swish']
intf_loss   = ['RelMSE']*ddfom.n_sub#['AbsMSE', 'AbsMSE']
intf_batch  = [32]*ddfom.n_sub#[32, 32]
wd_intf     = 1e-6

port_size   = [6]*len(ddfom.ports)#[5]
port_rnnz   = [5]*len(ddfom.ports)#[3]
port_rshift = [5]*len(ddfom.ports)#[5]
port_act    = ['Swish']*len(ddfom.ports)#['Swish']
port_loss   = ['RelMSE']*len(ddfom.ports)#['AbsMSE']
port_batch  = [32]*len(ddfom.ports)#[32, 32]
wd_port     = 1e-6

constraint_type = 'srpc'     # 'srpc' (strong rom-port constraints) or 'wfpc' (weak fom-port constraints)

# construct file and directory names for loading trained nets
net_dir = f'./trained_nets/nx_{nx}_ny_{ny}_nt_{nt}_visc_{viscosity}/DD_{nsub_x}x_by_{nsub_y}y/'

In [11]:
# load trained nets
intr_ae_list, intf_ae_list, port_ae_list = [], [], []
# wdstr = f'_wd{wd}' if wd > 0 else ''
for i in range(nsub):
    intr_file = f'ld_{intr_size[i]}_rnnz_{intr_rnnz[i]}_rshift_{intr_rshift[i]}_'  + \
                f'{intr_act[i]}_batch_{intr_batch[i]}_{intr_loss[i]}loss_wd{wd_intr}.p'
    intr_dir = net_dir+f'sub_{i+1}of{nsub}/interior/'
    intr_ae_list.append(torch.load(intr_dir+intr_file))

if constraint_type == 'wfpc':
    for i in range(nsub):
        intf_file = f'ld_{intf_size[i]}_rnnz_{intf_rnnz[i]}_rshift_{intf_rshift[i]}_'  + \
                    f'{intf_act[i]}_batch_{intf_batch[i]}_{intf_loss[i]}loss_wd{wd_intf}.p'
        intf_dir = net_dir+f'sub_{i+1}of{nsub}/interface/'
        intf_ae_list.append(torch.load(intf_dir+intf_file))
else:
    nports = len(ddfom.ports)
    for i in range(nports):
        port_file = f'ld_{port_size[i]}_rnnz_{port_rnnz[i]}_rshift_{port_rshift[i]}_'  + \
                    f'{port_act[i]}_batch_{port_batch[i]}_{port_loss[i]}loss_wd{wd_port}.p'
        port_dir = net_dir + f'port_{i+1}of{nports}/'
        port_ae_list.append(torch.load(port_dir+port_file))

In [12]:
# compute residual bases
ec_res     = 1e-16
nbasis_res = -1

res_bases = []
data_dir2 = data_dir + f'nx_{nx}_ny_{ny}_nt_{nt}_visc_{viscosity}/DD_{nsub_x}x_by_{nsub_y}y/' 
for i in range(ddfom.n_sub):
    sub_dir  = data_dir2 + f'sub_{i+1}of{ddfom.n_sub}/'
    res_dict  = pickle.load(open(sub_dir  + f'residual_svd_data.p', 'rb'))
    res_bases.append(compute_bases_from_svd(res_dict, ec=ec_res, nbasis=nbasis_res))
    
    print(f'Subdomain {i}:')
    print(f'residual_basis.shape  = {res_bases[i].shape}')

Subdomain 0:
residual_basis.shape  = (3600, 128)
Subdomain 1:
residual_basis.shape  = (3600, 131)


In [14]:
# NM-ROM parameters
hr              = True       # set to True to employ hyper reduction
hr_type         = 'collocation'
n_constraints   = 1          # number of constraints when using wfpc formulation
sample_ratio    = 2.0          # ratio of HR nodes to residual basis size
n_samples       = -1         # number of HR nodes to use. -1 uses sample_ratio
n_corners       = -1

# port_ae_list = [intf_ae_list[0]]
# constraint_type='srpc'

ddnmrom = DD_NMROM(ddfom, 
                   intr_ae_list, 
                   intf_ae_list=intf_ae_list,
                   port_ae_list=port_ae_list, 
                   res_bases=res_bases,
                   constraint_type=constraint_type, 
                   hr=hr,
                   hr_type=hr_type,
                   sample_ratio=sample_ratio,
                   n_samples=n_samples,
                   n_corners=n_corners,
                   n_constraints=n_constraints, 
                   seed=0,
                   scaling=-1)
ddnmrom.set_initial(lambda xy: u0(xy, mu), lambda xy: v0(xy, mu))

In [15]:
# train rbf interpolant
mu_list = [0.9, 0.95, 1.05, 1.1]
data_dir0 = 'data/'
data_dir1 = data_dir0 + f'nx_{nx}_ny_{ny}_nt_{nt}_visc_{viscosity}/'
snapshots = assemble_snapshot_matrix(data_dir1, mu_list)
rbfmdl    = dd_rbf(ddnmrom, t_lim, nt, mu_list, snapshots, n_snaps=(nt+1)//2) #, kernel='linear')

In [16]:
# compute intial guess using RBF interpolant
guess, rbftime = rbfmdl.compute_guess(t_lim, nt, mu, ddnmrom)

In [ ]:
march=nt
w_intr, w_intf, u_intr, v_intr, u_intf, v_intf, u_full, v_full, lam, romtime, ih, flag=\
ddnmrom.solve([0, 2/nt*march], march,
              guess=guess, 
              tol=1e-5, maxit=15, print_hist=False)
romtime += rbftime

In [ ]:
lt = np.array([np.linalg.norm(l) for l in lam])
plt.figure(figsize=(8,6))
plt.semilogy(lt+1e-16, 'o-')
plt.xlabel('$k$')
plt.ylabel('$\|\widehat{\mathbf{\lambda}}^{(k)}\|$')
plt.show()

## Compute error

In [ ]:
ntr    = u_full.shape[0]
abserr = np.zeros(ntr)
for i, s in enumerate(ddfom_data.subdomain):
    abserr += np.sum(np.square(s.interior.u[:ntr]-u_intr[i]), 1) + \
              np.sum(np.square(s.interior.v[:ntr]-v_intr[i]), 1) + \
              np.sum(np.square(s.interface.u[:ntr]-u_intf[i]), 1) + \
              np.sum(np.square(s.interface.v[:ntr]-v_intf[i]), 1)

abserr = np.max(np.sqrt(ddfom.hx*ddfom.hy*abserr/ddfom.n_sub))
print(f'Absolute error for {ntr-1} time steps = {abserr:1.3e}')
print(f'FOM runtime    = {ddfom_data.runtime:1.3e} seconds')
print(f'ROM runtime    = {romtime:1.3e} seconds')
print(f'Speedup        = {ddfom_data.runtime*ntr/nt/romtime}')

In [ ]:
Ax = np.zeros((march, ddnmrom.n_constraints))
for i, s in enumerate(ddnmrom.subdomain):
    for k in range(ntr-1):
        Ax[k] += s.constraint_mat@w_intf[i][k]
max_con = np.max(np.sqrt(np.sum(np.square(Ax),1)))
print(f'max_k ||Ax(t_k)||_2 = {max_con:1.2e}')

In [ ]:
Ax0 = np.zeros(ddnmrom.n_constraints)
for s in ddnmrom.subdomain:
    Ax0 += s.constraint_mat@s.interface.w0
print(f'initial constraint norm = {np.linalg.norm(Ax0):1.2e}')

## Compute residuals of encoded solution

In [ ]:
ht = (t_lim[1]-t_lim[0])/nt
lam = np.ones(ddnmrom.n_constraints)
lamfom = np.ones(ddfom.n_constraints)
residuals = np.zeros((ddfom.n_sub, nt+1))
# resfom_enc  = np.zeros((ddfom.n_sub, nt+1))
# resfom_ex   = np.zeros((ddfom.n_sub, nt+1))
constraints = np.zeros((nt+1, ddnmrom.n_constraints))
for i, s in enumerate(ddnmrom.subdomain):
    uv_intr = np.hstack([ddfom_data.subdomain[i].interior.u,
                         ddfom_data.subdomain[i].interior.v])
    uv_intf = np.hstack([ddfom_data.subdomain[i].interface.u,
                         ddfom_data.subdomain[i].interface.v])
    w_intr = s.interior.encoder.fwd(uv_intr)
    w_intf = s.interface.encoder.fwd(uv_intf)
    
#     uv_intr_dec = s.interior.decoder.fwd(w_intr)
#     u_intr_dec = uv_intr_dec[:, :s.interior.fomsize]
#     v_intr_dec = uv_intr_dec[:, s.interior.fomsize:]
    
#     uv_intf_dec = s.interface.decoder.fwd(w_intf)
#     u_intf_dec = uv_intf_dec[:, :s.interface.fomsize]
#     v_intf_dec = uv_intf_dec[:, s.interface.fomsize:]
    
    for k in range(nt):
        res, jac, H, rhs, Ag, Adg = s.res_jac(w_intr[k+1], w_intf[k+1], w_intr[k], w_intf[k], lam, ht)
        residuals[i, k] = np.sum(np.square(res))
        constraints[k] += Ag
        
#         rfenc = ddfom.subdomain[i].res_jac(u_intr_dec[k+1], v_intr_dec[k+1], u_intf_dec[k+1], v_intf_dec[k+1], 
#                                         u_intr_dec[k], v_intr_dec[k], u_intf_dec[k], v_intf_dec[k], lamfom, ht)[0]
#         resfom_enc[i, k] = np.sum(np.square(rfenc))
        
#         rfex = ddfom.subdomain[i].res_jac(ddfom_data.subdomain[i].interior.u[k+1], 
#                                           ddfom_data.subdomain[i].interior.v[k+1], 
#                                           ddfom_data.subdomain[i].interface.u[k+1], 
#                                           ddfom_data.subdomain[i].interface.v[k+1], 
#                                           ddfom_data.subdomain[i].interior.u[k], 
#                                           ddfom_data.subdomain[i].interior.v[k], 
#                                           ddfom_data.subdomain[i].interface.u[k], 
#                                           ddfom_data.subdomain[i].interface.v[k], 
#                                           lamfom, ht)[0]
#         resfom_ex[i, k] = np.sum(np.square(rfex))

In [ ]:
tt=np.linspace(t_lim[0], t_lim[1], nt+1)
con_norm = np.sqrt(np.sum(np.square(constraints), 1))
plt.figure(figsize=(8,6))
plt.semilogy(tt,con_norm+1e-16)
plt.xlabel('$t_k$')
plt.ylabel('$\|\Sigma_i \widehat{\mathbf{A}}_i \widehat{\mathbf{x}}_i^{\Gamma(k)}\|_2$')
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
ls = ['-', '--', '.-', ':']
for i in range(ddnmrom.n_sub):
    plt.semilogy(tt, residuals[i], ls[i], label=f'Subdomain {i}')
plt.xlabel('$t_k$')
plt.ylabel('$\|\mathbf{r}_i(\mathbf{g}_i^\Omega(\mathbf{h}_i^\Omega(\mathbf{x}_i^{\Omega(k)})), \mathbf{g}_i^\Gamma(\mathbf{h}_i^\Gamma(\mathbf{x}_i^{\Gamma (k)})))\|_2^2$')
plt.legend()
plt.show()

## Save solution, gifs of solution

In [ ]:
# store snapshots
con_dir = 'srpc/' if constraint_type == 'srpc' else f'wfpc_ncon_{n_constraints}'
intr_str = f'intr_ld{intr_size}rnz{intr_rnnz}rs{intr_rshift}_'
if constraint_type == 'srpc':
    intf_str = f'port_ld{port_size}rnz{port_rnnz}rs{port_rshift}_'
else:
    intf_str = f'intf_ld{intf_size}rnz{intf_rnnz}rs{intf_rshift}_'
if hr:
    hr_str = f'hr_{n_samples}_' if n_samples > 0 else f'hr_{sample_ratio}x_'
else:
    hr_str = ''
    
# print('\nSaving data...')
# for i, s in enumerate(ddnmrom.subdomain):
#     hr_str = f'hr_{s.hr_nodes.size}_' if hr else ''
#     sub_dir  = data_dir2 + f'sub_{i+1}of{ddnmrom.n_sub}/'
#     intr_dir = sub_dir + 'interior/' + con_dir
#     intf_dir = sub_dir + 'interface/' + con_dir
#     for d in [intr_dir, intf_dir]:
#         if not os.path.exists(d): os.mkdir(d)
            
#     intr_dict = {'solution': w_intr[i],
#                  'runtime':  romtime}
#     intf_dict = {'solution': w_intf[i],
#                  'runtime':  romtime}

#     intr_filename = intr_dir + f'nmrom_{intr_str}{hr_str}mu_{mu}_state.p'
#     intf_filename = intf_dir + f'nmrom_{intf_str}{hr_str}mu_{mu}_state.p'

#     pickle.dump(intr_dict, open(intr_filename, 'wb'))
#     pickle.dump(intf_dict, open(intf_filename, 'wb'))
# print('Done!')    

In [ ]:
# frame updater for animation
umin = np.min([s.interior.u.min() for s in ddfom_data.subdomain]+
              [s.interface.u.min() for s in ddfom_data.subdomain])
umax = np.max([s.interior.u.max() for s in ddfom_data.subdomain]+
              [s.interface.u.max() for s in ddfom_data.subdomain])
vmin = np.min([s.interior.v.min() for s in ddfom_data.subdomain]+
              [s.interface.v.min() for s in ddfom_data.subdomain])
vmax = np.max([s.interior.v.max() for s in ddfom_data.subdomain]+
              [s.interface.v.max() for s in ddfom_data.subdomain])

X, Y = np.meshgrid(fom.xx, fom.yy)
def update_frame(i, Z, zmin, zmax, cb_label):
    plt.clf()
    plt.pcolormesh(X, Y, Z[i], cmap='viridis', shading='auto', vmin=zmin, vmax=zmax) 
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.title('$t_{'+f'{i:4d}'+'}' + f'={ht*i+t_lim[0]:1.4f}$')
    cb = plt.colorbar(orientation='vertical', label=cb_label)
    return plt

In [ ]:
# save gifs of solutions
con_dir = 'srpc/' if constraint_type == 'srpc' else f'wfpc_ncon_{n_constraints}'
for suffix in ['', con_dir]:
    rom_figs = fig_dir0 + \
               f'nx_{nx}_ny_{ny}_nt_{nt}_visc_{viscosity}/' +\
               f'mu_{mu}/ddnmrom_{nsub_x}x_by_{nsub_y}y/' + suffix
    if not os.path.exists(rom_figs): os.mkdir(rom_figs)

prefix = rom_figs + f'{intr_str}{intf_str}{hr_str}_ntr{ntr}_'

print('\nGenerating gif for u state...')
UU = u_full.reshape(u_full.shape[0], ddfom.ny, ddfom.nx)
fig = plt.figure()
ani = animation.FuncAnimation(fig, lambda i: update_frame(i, UU, umin, umax, '$u(x,y)$'),
                              frames=u_full.shape[0], interval=1)
ani.save(prefix + 'u_state.gif', writer='imagemagick', fps=nt//10)
plt.close()
print('Done!')
print('u state gif = ' + prefix + 'u_state.gif')
Image(filename=prefix+'u_state.gif')

In [ ]:
print('\nGenerating gif for v state...')
VV = v_full.reshape(v_full.shape[0], ddfom.ny, ddfom.nx)
fig = plt.figure()
ani = animation.FuncAnimation(fig, lambda i: update_frame(i, VV, vmin, vmax, '$v(x,y)$'),
                              frames=v_full.shape[0], interval=1)
ani.save(prefix + f'v_state_nt{ntr}.gif', writer='imagemagick', fps=nt//10)
plt.close()
print('Done!')
print('v state gif = ' + prefix + 'v_state.gif')
Image(filename=prefix+f'v_state_nt{ntr}.gif')

## Sandbox